In [1]:
from models.models_one_task import AlexNet, vgg
from datasets.ramas import RamasDataset
from constants import *
from torchsummary import summary
import torch
import torch.nn as nn
import skorch
from skorch.helper import predefined_split
from skorch.dataset import Dataset
from skorch.classifier import NeuralNetClassifier
import skorch.callbacks as callbacks
import numpy as np
import matplotlib.pyplot as plt
import os
import pickle

In [2]:
ramas_224_train = RamasDataset(RAMAS_PATH_TO_WAVS, 'Ramas224Descrete',
                 spectrogram_shape=224,
                 augmentation=True, padding='repeat', mode='train',  tasks='emotion', type='descrete')
ramas_224_test = RamasDataset(RAMAS_PATH_TO_WAVS, 'Ramas224Descrete',
                 spectrogram_shape=224,
                 augmentation=False, padding='repeat', mode='test',  tasks='emotion', type='descrete')

============= INITIALIZING DATASET Ramas224Descrete_224_train ===============
============================ SUCCESS! =========================
============= INITIALIZING DATASET Ramas224Descrete_224_test ===============
============================ SUCCESS! =========================


In [3]:
train_dataset = ramas_224_train
test_dataset = ramas_224_test
filename = 'AlexNet--{}_augmentation-{}.md'.format(train_dataset.name, str(train_dataset.augmentation).lower())
best_model_file_path = os.path.join(RESULTS_FOLDER, filename)
callback_train_acc = callbacks.EpochScoring(scoring="accuracy", 
                                            lower_is_better=False, 
                                            on_train=True, 
                                            name='train_acc')
callback_save_best = callbacks.Checkpoint(monitor='valid_loss_best', 
                                          f_params=None, 
                                          f_optimizer=None, 
                                          f_criterion=None, 
                                          f_history=None, 
                                          f_pickle=best_model_file_path,  
                                          event_name='event_cp')
callback_early_stop = callbacks.EarlyStopping(monitor='valid_loss', patience=30, 
                                              threshold_mode='rel', lower_is_better=True)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [4]:
model = AlexNet(num_classes=8)
net = skorch.classifier.NeuralNetClassifier(
    model, criterion=nn.CrossEntropyLoss, optimizer=torch.optim.Adam,
    lr=3e-4, max_epochs=300, batch_size=32, train_split=predefined_split(test_dataset), 
    device=device, iterator_train__shuffle=True, 
    callbacks=[
        callback_train_acc,
        callback_save_best,
        callback_early_stop
    ]
)

In [5]:
net.fit(train_dataset, y=None)

  epoch    train_acc    train_loss    valid_acc    valid_loss    cp      dur
-------  -----------  ------------  -----------  ------------  ----  -------
      1       0.2073        1.9601       0.2264        1.9107     +  46.6378
      2       0.2225        1.8998       0.2869        1.7789     +  46.1199
      3       0.2647        1.8206       0.2623        1.8543        46.0725
      4       0.2673        1.7855       0.2992        1.7316     +  46.0463
      5       0.2855        1.7609       0.3012        1.7265     +  46.1071
      6       0.2924        1.7486       0.3053        1.7133     +  46.1171
      7       0.2916        1.7321       0.3094        1.6991     +  46.0627
      8       0.3006        1.7136       0.3012        1.6932     +  46.0935
      9       0.3057        1.7159       0.3197        1.6835     +  46.0566
     10       0.3096        1.6948       0.3238        1.6952        46.0964
     11       0.3055        1.6936       0.2869        1.7072        46.0293

<class 'skorch.classifier.NeuralNetClassifier'>[initialized](
  module_=AlexNet(
    (features): Sequential(
      (0): Conv2d(1, 64, kernel_size=(11, 11), stride=(4, 4), padding=(2, 2))
      (1): ReLU(inplace=True)
      (2): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
      (3): Conv2d(64, 192, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
      (4): ReLU(inplace=True)
      (5): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
      (6): Conv2d(192, 384, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (7): ReLU(inplace=True)
      (8): Conv2d(384, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (9): ReLU(inplace=True)
      (10): Conv2d(256, 256, kernel_size=(4, 4), stride=(1, 1), padding=(1, 1))
      (11): ReLU(inplace=True)
      (12): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
    )
    (avgpool): AdaptiveAvgPool2d(output_size=(5, 5))
    (classifier): Sequential

In [3]:
train_dataset = ramas_224_train
test_dataset = ramas_224_test
filename = 'VGGNet--{}_augmentation-{}.md'.format(train_dataset.name, str(train_dataset.augmentation).lower())
best_model_file_path = os.path.join(RESULTS_FOLDER, filename)
callback_train_acc = callbacks.EpochScoring(scoring="accuracy", 
                                            lower_is_better=False, 
                                            on_train=True, 
                                            name='train_acc')
callback_save_best = callbacks.Checkpoint(monitor='valid_loss_best', 
                                          f_params=None, 
                                          f_optimizer=None, 
                                          f_criterion=None, 
                                          f_history=None, 
                                          f_pickle=best_model_file_path,  
                                          event_name='event_cp')
callback_early_stop = callbacks.EarlyStopping(monitor='valid_loss', patience=30, 
                                              threshold_mode='rel', lower_is_better=True)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [4]:
model = vgg(type=11, bn=True, num_classes=8)
net = skorch.classifier.NeuralNetClassifier(
    model, criterion=nn.CrossEntropyLoss, optimizer=torch.optim.Adam,
    lr=3e-4, max_epochs=300, batch_size=32, train_split=predefined_split(test_dataset), 
    device=device, iterator_train__shuffle=True, 
    callbacks=[
        callback_train_acc,
        callback_save_best,
        callback_early_stop
    ]
)

In [5]:
net.fit(train_dataset, y=None)

  epoch    train_acc    train_loss    valid_acc    valid_loss    cp      dur
-------  -----------  ------------  -----------  ------------  ----  -------
      1       0.2365        1.9116       0.1742        3.1456     +  67.4569
      2       0.2704        1.7873       0.3166        1.7154     +  67.3093
      3       0.2940        1.7394       0.2859        1.6974     +  67.3468
      4       0.3091        1.7186       0.2213        2.3580        67.5020
      5       0.3209        1.7093       0.2982        1.6944     +  67.5118
      6       0.3278        1.6698       0.2736        2.4418        67.2240
      7       0.3547        1.6653       0.3463        1.6413     +  67.3927
      8       0.3598        1.6426       0.3443        1.6632        67.3277
      9       0.3865        1.6186       0.3811        1.6149     +  67.1798
     10       0.3975        1.5942       0.3166        1.9065        67.3908
     11       0.4044        1.5757       0.3822        1.7001        67.1412

<class 'skorch.classifier.NeuralNetClassifier'>[initialized](
  module_=VGG(
    (features): Sequential(
      (0): Conv2d(1, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (2): ReLU(inplace=True)
      (3): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
      (4): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (5): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (6): ReLU(inplace=True)
      (7): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
      (8): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (9): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (10): ReLU(inplace=True)
      (11): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (12): BatchNorm2d(256, eps=1e-05, mome

In [2]:
ramas_224_train = RamasDataset(RAMAS_PATH_TO_WAVS, 'Ramas224Binary',
                 spectrogram_shape=224,
                 augmentation=True, padding='repeat', mode='train',  tasks='emotion', type='binary')
ramas_224_test = RamasDataset(RAMAS_PATH_TO_WAVS, 'Ramas224Binary',
                 spectrogram_shape=224,
                 augmentation=False, padding='repeat', mode='test',  tasks='emotion', type='binary')

============= INITIALIZING DATASET Ramas224Binary_224_train ===============
============================ SUCCESS! =========================
============= INITIALIZING DATASET Ramas224Binary_224_test ===============
============================ SUCCESS! =========================


In [3]:
train_dataset = ramas_224_train
test_dataset = ramas_224_test
filename = 'AlexNet--{}_augmentation-{}.md'.format(train_dataset.name, str(train_dataset.augmentation).lower())
best_model_file_path = os.path.join(RESULTS_FOLDER, filename)
callback_train_acc = callbacks.EpochScoring(scoring="accuracy", 
                                            lower_is_better=False, 
                                            on_train=True, 
                                            name='train_acc')
callback_save_best = callbacks.Checkpoint(monitor='valid_loss_best', 
                                          f_params=None, 
                                          f_optimizer=None, 
                                          f_criterion=None, 
                                          f_history=None, 
                                          f_pickle=best_model_file_path,  
                                          event_name='event_cp')
callback_early_stop = callbacks.EarlyStopping(monitor='valid_loss', patience=20, 
                                              threshold_mode='rel', lower_is_better=True)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [4]:
model = AlexNet(num_classes=2)
net = skorch.classifier.NeuralNetClassifier(
    model, criterion=nn.CrossEntropyLoss, optimizer=torch.optim.Adam,
    lr=1e-5, max_epochs=300, batch_size=32, train_split=predefined_split(test_dataset), 
    device=device, iterator_train__shuffle=True, 
    callbacks=[
        callback_train_acc,
        callback_save_best,
#         callback_early_stop
    ]
)

In [5]:
net.fit(train_dataset, y=None)

  epoch    train_acc    train_loss    valid_acc    valid_loss    cp      dur
-------  -----------  ------------  -----------  ------------  ----  -------
      1       0.7873        0.5056       0.8299        0.4718     +  46.1976
      2       0.8298        0.4650       0.8299        0.4591     +  45.6342
      3       0.8298        0.4555       0.8299        0.4635        46.3378
      4       0.8298        0.4433       0.8299        0.4377     +  45.6760
      5       0.8298        0.4338       0.8299        0.4388        45.5918
      6       0.8298        0.4266       0.8299        0.4275     +  45.7920
      7       0.8298        0.4190       0.8299        0.4201     +  45.9176
      8       0.8298        0.4109       0.8299        0.4131     +  45.8273
      9       0.8301        0.4058       0.8299        0.4110     +  46.0009
     10       0.8306        0.4009       0.8320        0.4057     +  45.6033
     11       0.8321        0.3938       0.8350        0.4017     +  45.8485

<class 'skorch.classifier.NeuralNetClassifier'>[initialized](
  module_=AlexNet(
    (features): Sequential(
      (0): Conv2d(1, 64, kernel_size=(11, 11), stride=(4, 4), padding=(2, 2))
      (1): ReLU(inplace=True)
      (2): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
      (3): Conv2d(64, 192, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
      (4): ReLU(inplace=True)
      (5): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
      (6): Conv2d(192, 384, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (7): ReLU(inplace=True)
      (8): Conv2d(384, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (9): ReLU(inplace=True)
      (10): Conv2d(256, 256, kernel_size=(4, 4), stride=(1, 1), padding=(1, 1))
      (11): ReLU(inplace=True)
      (12): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
    )
    (avgpool): AdaptiveAvgPool2d(output_size=(5, 5))
    (classifier): Sequential

In [6]:
train_dataset = ramas_224_train
test_dataset = ramas_224_test
filename = 'VGGNet--{}_augmentation-{}.md'.format(train_dataset.name, str(train_dataset.augmentation).lower())
best_model_file_path = os.path.join(RESULTS_FOLDER, filename)
callback_train_acc = callbacks.EpochScoring(scoring="accuracy", 
                                            lower_is_better=False, 
                                            on_train=True, 
                                            name='train_acc')
callback_save_best = callbacks.Checkpoint(monitor='valid_loss_best', 
                                          f_params=None, 
                                          f_optimizer=None, 
                                          f_criterion=None, 
                                          f_history=None, 
                                          f_pickle=best_model_file_path,  
                                          event_name='event_cp')
callback_early_stop = callbacks.EarlyStopping(monitor='valid_loss', patience=20, 
                                              threshold_mode='rel', lower_is_better=True)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [7]:
model = vgg(type=11, bn=True, num_classes=2)
net = skorch.classifier.NeuralNetClassifier(
    model, criterion=nn.CrossEntropyLoss, optimizer=torch.optim.Adam,
    lr=1e-5, max_epochs=300, batch_size=32, train_split=predefined_split(test_dataset), 
    device=device, iterator_train__shuffle=True, 
    callbacks=[
        callback_train_acc,
        callback_save_best,
#         callback_early_stop
    ]
)

In [8]:
net.fit(train_dataset, y=None)

  epoch    train_acc    train_loss    valid_acc    valid_loss    cp      dur
-------  -----------  ------------  -----------  ------------  ----  -------
      1       0.8291        0.4510       0.8299        0.4352     +  67.4910
      2       0.8296        0.4287       0.8279        0.4298     +  67.6558
      3       0.8309        0.4207       0.8299        0.4125     +  67.5511
      4       0.8334        0.4009       0.8443        0.4138        67.7493
      5       0.8385        0.3918       0.8453        0.4003     +  67.7085
      6       0.8385        0.3825       0.8381        0.4027        67.6332
      7       0.8444        0.3754       0.8258        0.4088        67.7724
      8       0.8429        0.3694       0.8217        0.4211        67.6102
      9       0.8526        0.3563       0.8596        0.3851     +  67.7306
     10       0.8567        0.3489       0.8443        0.4391        67.7611
     11       0.8590        0.3490       0.8525        0.3940        67.7689

<class 'skorch.classifier.NeuralNetClassifier'>[initialized](
  module_=VGG(
    (features): Sequential(
      (0): Conv2d(1, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (2): ReLU(inplace=True)
      (3): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
      (4): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (5): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (6): ReLU(inplace=True)
      (7): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
      (8): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (9): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (10): ReLU(inplace=True)
      (11): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (12): BatchNorm2d(256, eps=1e-05, mome